In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [2]:
cd D:\New folder\hr_analytics

D:\New folder\hr_analytics


In [3]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [4]:
print(train_df.shape)
train_df.head()

(18359, 14)


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,23798,city_149,0.689,Male,Has relevent experience,no_enrollment,Graduate,STEM,3,100-500,Pvt Ltd,1,106,0
1,29166,city_83,0.923,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,<10,Funded Startup,1,69,0
2,46,city_16,0.910,NaN,Has relevent experience,no_enrollment,Graduate,STEM,6,50-99,Public Sector,2,4,0
3,18527,city_64,0.666,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,50-99,Pvt Ltd,1,26,0
4,21751,city_100,0.887,NaN,No relevent experience,no_enrollment,Masters,STEM,8,NaN,NaN,2,88,1


In [5]:
lbls=train_df['target']
train_df=train_df.drop(['enrollee_id','target'],axis=1)

In [6]:
train_df.shape

(18359, 12)

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18359 entries, 0 to 18358
Data columns (total 12 columns):
city                      18359 non-null object
city_development_index    18359 non-null float64
gender                    14261 non-null object
relevent_experience       18359 non-null object
enrolled_university       18017 non-null object
education_level           17902 non-null object
major_discipline          15521 non-null object
experience                18300 non-null object
company_size              13580 non-null object
company_type              13320 non-null object
last_new_job              17992 non-null object
training_hours            18359 non-null int64
dtypes: float64(1), int64(1), object(10)
memory usage: 1.7+ MB


In [8]:
otype_cols=['city','gender','relevent_experience','enrolled_university','education_level','major_discipline','company_type','company_size','last_new_job']
rem_cols=['city_development_index','training_hours','experience']

In [9]:
for col in train_df.columns:
    a=train_df[col].describe()
    if a.dtype=='O':
        train_df[col]=train_df[col].fillna(a.top)
        test_df[col]=test_df[col].fillna(a.top)

In [10]:
train_df[train_df['experience']=='>20']='25'
train_df[train_df['experience']=='<1']='0'
train_df['experience']=train_df['experience'].astype(float)

In [11]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
X=pd.DataFrame()

In [12]:
les={}
ohes={}
for col in otype_cols:
    le=LabelEncoder()
    int_enc=le.fit_transform(train_df[col])
    ohe=OneHotEncoder(sparse=False,categories='auto')
    x=ohe.fit_transform(int_enc.reshape(-1,1))
    les[col]=le
    ohes[col]=ohe
    for i in range(x.shape[1]):
        col_name=col+str(i)
        X[col_name]=x[:,i]
    

In [13]:
for col in rem_cols:
    X[col]=train_df[col].astype(float)

In [14]:
X.shape

(18359, 183)

In [15]:
x_train,x_test,y_train,y_test=train_test_split(X,lbls,test_size=0.4,shuffle=True,stratify=lbls,random_state=21)

In [16]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(x_train,y_train)
lr.score(x_test,y_test)

C:\Users\naren\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8679193899782135

In [17]:
from sklearn.linear_model import RidgeClassifier
ridge=RidgeClassifier()
ridge.fit(x_train,y_train)
ridge.score(x_test,y_test)

0.8672385620915033

In [ ]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(x_train,y_train)
svm.score(x_test,y_test)

In [18]:
from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(max_depth=5)
tree.fit(x_train,y_train)
tree.score(x_test,y_test)

0.866557734204793

In [19]:
from sklearn.ensemble import RandomForestClassifier
rdf=RandomForestClassifier(n_estimators=250,random_state=21)
rdf.fit(x_train,y_train)
rdf.score(x_train,y_train)

0.9751248297775761

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gbc= GradientBoostingClassifier()
gbc.fit(x_train,y_train)
gbc.score(x_test,y_test)

In [ ]:
from xgboost import XGBClassifier
xgb=XGBClassifier()
xgb.fit(x_train,y_train)
xgb.score(x_test,y_test)

In [20]:
e_id=test_df['enrollee_id'].astype(int)

In [21]:
test_df[test_df['experience']=='>20']='25'
test_df[test_df['experience']=='<1']='0'
#test_df['experience']=test_df['experience'].astype(float)

In [22]:
X_test=pd.DataFrame()

In [23]:
for col in otype_cols:
    #le=LabelEncoder()
    int_enc=les[col].transform(test_df[col])
    #ohe=OneHotEncoder(sparse=False,categories='auto')
    x=ohes[col].transform(int_enc.reshape(-1,1))
    
    for i in range(x.shape[1]):
        col_name=col+str(i)
        X_test[col_name]=x[:,i]

In [24]:
for col in rem_cols:
    X_test[col]=test_df[col].astype(float)

In [25]:
X_test.shape

(15021, 183)

In [51]:
y_pred=rdf.predict_proba(X_test)

In [55]:
req_prob=y_pred[:,1]

In [56]:
preds=pd.DataFrame({'enrolleeid':e_id,'prediction':req_prob})

In [58]:
preds.head()

,enrolleeid,prediction
0,16548,0.230962
1,12036,0.098724
2,11061,0.224000
3,5032,0.098724
4,17599,0.048000


In [59]:
preds['prediction'].describe()

count    15021.000000
mean         0.136400
std          0.111774
min          0.000000
25%          0.068000
50%          0.098724
75%          0.176000
max          0.896000
Name: prediction, dtype: float64

In [60]:
preds.to_csv('randomforestmodel.csv')